In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import random
# from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = "G:/Gallblader Diseases Dataset/train"
test_dir = "G:/Gallblader Diseases Dataset/test"
val_dir = "G:/Gallblader Diseases Dataset/val"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

input_shape=(64,64)

datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)
datagen_val=ImageDataGenerator(rescale=1./255)
batch_size = 64

# datagen_train = ImageDataGenerator(rescale=1./255, validation_split=0.20)

# generator_train = datagen_train.flow_from_directory(directory=train_dir,
#                                                     target_size=input_shape,
#                                                     batch_size=batch_size,
#                                                     subset="training",
#                                                     shuffle=True)
# generator_val = datagen_train.flow_from_directory(directory=train_dir,
#                                                   target_size=input_shape,
#                                                   batch_size=batch_size,
#                                                   subset="validation",
#                                                   shuffle=True)


generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_val = datagen_val.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

In [ ]:
input_shape = (64, 64, 3)
num_classes=9

In [ ]:
learning_rate = 0.0001
weight_decay = 0.00001
batch_size = 128
num_epochs = 50
image_size = 64  # We'll resize input images to this size
patch_size = 3  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 32
num_heads = 2
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [1024, 512]  # Size of the dense layers of the final classifier

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # # Augment data.
    # augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
        ],
    )

    # callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    history = model.fit(
        generator_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=generator_val,
        # callbacks=[callback]
    )

    return history



In [ ]:
vit_classifier.summary()

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
# plt.title('Vision Transformer Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc = 'upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
# plt.title('Vision Transformer Training Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc = 'lower right')
plt.show()

In [ ]:
import numpy as np
predict = vit_classifier.predict(generator_test)
predict=np.argmax(predict,axis=1)
# y_pred = vit_classifier.predict(generator_test)
# y_pred=np.argmax(y_pred,axis=1)

In [ ]:
generator_test.class_indices.keys()

In [ ]:
from sklearn.metrics import classification_report

clf_report_df = pd.DataFrame(classification_report(generator_test.classes, predict, target_names=['Gallstones', 'Abdomen and retroperitoneum', 'cholecystitis', 'Membranous and gangrenous cholecystitis', 'Perforation', 'Polyps and cholesterol crystals', 'Adenomyomatosis', 'Carcinoma', 'Various causes of gallbladder wall thickening'], output_dict=True, digits=4)).T

sns.heatmap(clf_report_df.iloc[:-1, :-1], annot=True, fmt=".4f", cmap=sns.cubehelix_palette(as_cmap=True))

In [ ]:
# pip install sklearn-evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
array = confusion_matrix(generator_test.classes, predict)
df_cm = pd.DataFrame(array, index = ['Gallstones', 'Abdomen and retroperitoneum', 'cholecystitis', 'Membranous and gangrenous cholecystitis', 'Perforation', 'Polyps and cholesterol crystals', 'Adenomyomatosis', 'Carcinoma', 'Various causes of gallbladder wall thickening'],
                  columns = ['Gallstones', 'Abdomen and retroperitoneum', 'cholecystitis', 'Membranous and gangrenous cholecystitis', 'Perforation', 'Polyps and cholesterol crystals', 'Adenomyomatosis', 'Carcinoma', 'Various causes of gallbladder wall thickening'])
plt.figure(figsize = (6,5))
cm_plot = sns.heatmap(df_cm, annot=True, cmap='GnBu', fmt='g')
cm_plot.set_xlabel('Predicted Labels')
cm_plot.set_ylabel('True Labels')
cm_plot.set_title('Confusion Matrix From Self-Attention Based VIT', size=12)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

y_test=generator_test.classes
y_pred = predict
# y_pred=np.argmax(y_pred,axis=1)

target= ['Gallstones', 'Abdomen and retroperitoneum', 'cholecystitis', 'Membranous and gangrenous cholecystitis', 'Perforation', 'Polyps and cholesterol crystals', 'Adenomyomatosis', 'Carcinoma', 'Various causes of gallbladder wall thickening']

# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 10))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, lw=2, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
    return roc_auc_score(y_test, y_pred, average=average)


print('ROC AUC score:', multiclass_roc_auc_score(y_test, y_pred))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
# c_ax.set_title('ROC Curve for nct-crc-he-100k Dataset ', size=12)
plt.show()